## Training Material Outline: SAR Data Processing and Change Detection for Dam Monitoring (Yamba Dam Case Study)

This training material will equip you with the skills to process SAR data, apply noise reduction techniques, visualize dam water extents, and perform change detection analyses for dam monitoring, seasonal variation studies, and water resource management.

**Module 1: Introduction to SAR and ALOS-2 Data**

* **1.1 What is SAR?** (Brief explanation of Synthetic Aperture Radar principles and advantages for dam monitoring)

Synthetic Aperture Radar (SAR) is a remote sensing technique that uses radar signals to create images of the Earth's surface. Unlike optical sensors, SAR can penetrate clouds and operate day and night, making it ideal for dam monitoring, especially in frequently cloud-covered regions. SAR's sensitivity to surface roughness and moisture content allows for detailed mapping of water extents and surrounding terrain.

* **1.2 Introduction to ALOS-2:**

    * Overview of the ALOS-2 satellite mission (Reference: 8.1.1.a, 8.1.1.b)
    * Sensor specifications and capabilities (Reference: 8.1.1.a, 8.1.1.b)
    * Key applications of ALOS-2 data, with a focus on water resource management (Reference: 8.1.1.c, Module 8 intro)
    * Limitations of ALOS-2 data for water detection (Reference: 8.2.1.1.a, Table 8.2.1.1.2)

ALOS-2 (Advanced Land Observing Satellite-2), also known as DAICHI-2, is a Japanese Earth observation satellite equipped with the PALSAR-2 L-band SAR sensor.  This sensor provides high-resolution imagery useful for various applications, including disaster monitoring, land use mapping, and resource exploration.  In water resource management, ALOS-2 data can be used to monitor dam water levels, assess flood inundation, and map changes in water bodies over time. However, limitations such as terrain shadowing and misclassification of smooth surfaces like bare land can affect water detection accuracy.

* **1.3 Yamba Dam Case Study:**
    * Geographic location and characteristics of Yamba Dam
    * Importance of monitoring Yamba Dam's water extent

Yamba Dam is located in Japan and plays a crucial role in water supply, flood control, and irrigation. Monitoring its water extent is essential for effective water resource management, especially in the face of climate change and extreme weather events. Accurate monitoring enables informed decisions regarding water allocation, flood mitigation, and drought preparedness.

**Module 2: SAR Data Preprocessing and Visualization**

* **2.1 Data Acquisition and Format:**
    * Accessing ALOS-2 data (Reference: Appendix)
    * Understanding GeoTIFF format (Reference: 8.1.2.a, 8.1.2.f)
    * Loading ALOS-2 data in Python using Rasterio (Reference: 8.1, 9.1.2, Download file from Owncloud directly)

In [ ]:
!pip install rasterio matplotlib

import rasterio
import matplotlib.pyplot as plt

# Replace with your actual file path
alos_file = 'example.tif'

with rasterio.open(alos_file) as src:
    alos_data = src.read(1)
    plt.imshow(alos_data, cmap='gray')
    plt.show()

* **2.2 Speckle Noise Reduction:**
    * Understanding speckle noise in SAR imagery (Reference: 8.2, Lee filter sections)
    * Implementing the Lee filter in Python for speckle reduction (Reference: 8.2, Lee filter sections)
    * Visualizing the effects of speckle filtering (Reference: 8.2, Lee filter sections)

In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter

def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2
    overall_variance = np.var(img)
    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

filtered_alos = lee_filter(alos_data, 5) # Example size 5x5
plt.imshow(filtered_alos, cmap='gray')
plt.show()

* **2.3 Visualizing SAR Data:**
    * Basic visualization techniques using Matplotlib (Reference: 8.1, 9.1.2, Download file from Owncloud directly, 9.1.2 code examples)
    * Applying colormaps for enhanced visualization (Reference: 8.1, Download file from Owncloud directly)
    * Generating histograms of pixel values (Reference: 8.1, Download file from Owncloud directly)

In [ ]:
plt.imshow(alos_data, cmap='viridis')
plt.colorbar()
plt.show()

plt.hist(alos_data.flatten(), bins=256)
plt.show()

**Module 3: Dam Water Extent Extraction**

* **3.1 Principles of Water Detection in SAR:**
    * SAR backscatter characteristics of water (Reference: 8.2.1.1.a)
    * Differentiating water from other land cover types in SAR imagery (Reference: 8.2.1.1.a, Table 8.2.1.1.1)

Water typically appears dark in SAR imagery due to its smooth surface, which reflects radar signals away from the sensor. This low backscatter contrast with higher backscatter from rougher surfaces like vegetation or urban areas, allowing for differentiation.

* **3.2 Otsu's Thresholding Method:**
    * Introduction to Otsu's method for image segmentation (Reference: 8.2.1.2.a, 8.2.1.2.b)
    * Implementing Otsu's method in Python using scikit-image (Reference: 8.2.1.2.c)

In [ ]:
from skimage.filters import threshold_otsu

otsu_threshold = threshold_otsu(filtered_alos)
water_mask = filtered_alos < otsu_threshold
plt.imshow(water_mask)
plt.show()

* **3.3 Applying Otsu's Method to ALOS-2 Data:**
    * Hands-on exercise: Extracting water extent at Yamba Dam using Otsu's method (Reference: 8.2, Using Otsu's method to classify water area)
    * Refining water masks using median filtering (Reference: 8.2, Remove noise section)
    * Calculating water area (Reference: 8.2, Calculate water area section)
    * (Optional) Converting raster to vector format (Reference: 8.2, Convert raster to vector section)

In [ ]:
from scipy.ndimage import median_filter

refined_mask = median_filter(water_mask, size=3) # Example 3x3 filter

# Assuming pixel resolution is known (e.g., 10m x 10m)
pixel_area = 100  # square meters
water_area = np.sum(refined_mask) * pixel_area / 10000 # hectares
print(f"Water area: {water_area} hectares")

* **3.4 Integrating Slope Data (for Mountainous Areas):**
    * Importance of slope data for accurate water detection in mountainous terrain (Reference: 8.2, Using Otsu's method and slope data to classify water area)
    * Hands-on exercise: Combining Otsu's method with slope data for improved classification (Reference: 8.2, Using Otsu's method and slope data to classify water area)

In mountainous areas, slope data helps to differentiate water bodies from shadowed regions, which can have similar low backscatter in SAR imagery. By masking out steep slopes, the accuracy of water detection can be improved.

**Module 4: Change Detection Analysis**

* **4.1 Introduction to Time-Series Analysis:** (Reference: 8.3.1.a)

Time-series analysis involves analyzing data collected over time to identify trends, patterns, and changes. In dam monitoring, this can be used to track water level fluctuations, assess seasonal variations, and detect long-term changes in water availability.

* **4.2 Preparing Time-Series Data:**
    * Acquiring multi-temporal ALOS-2 data (Reference: 8.3.1.b, Appendix)
    * Calibrating and filtering multi-temporal data (Reference: 8.3.1.b, 9.3, Prepare time-series data from ALOS-2 imagery section)
    * Creating a stacked image with multiple time points (Reference: 8.3.1.b, Layer stacking section, 9.3, Apply Layer Stacking between 2007-2023 section)

Multi-temporal ALOS-2 data from different acquisition dates needs to be preprocessed (calibrated and filtered) and stacked into a single multi-band image for change detection analysis.

* **4.3 Visualizing Change:**
    * Using QGIS Temporal/Spectral Profile tool to visualize changes at specific locations (Reference: 8.3.1.c)
    * Generating composite images for visual comparison (Reference: 8.3.1.c)

Changes in water extent can be visualized using tools like the Temporal/Spectral Profile in QGIS, which shows changes in pixel values over time at a specific location. Composite images, created by combining different time points, provide a visual comparison of changes.

* **4.4 Interpreting Change Detection Results:**
    * Analyzing temporal profiles and composite images to understand water extent changes (Reference: 8.3.2, 9.3, How to visualize and interpret the results for specific change locations section)
    * Examples of interpreting changes at Yamba Dam due to typhoon Hagibis (Reference: 8.3.2)

Interpreting change detection results involves analyzing temporal profiles and composite images to understand how water extent has changed over time. For example, a decrease in water extent after a typhoon might indicate increased water usage or evaporation.

* **4.5 (Optional) Accuracy Assessment:** (Reference: 8.2.1.3)
    * Introduction to accuracy assessment and confusion matrix
    * Example of calculating accuracy for water extent classification

Accuracy assessment involves comparing the classified water extent with ground truth data to evaluate the accuracy of the classification. A confusion matrix summarizes the classification results, showing the number of true positives, true negatives, false positives, and false negatives.

**Conclusion and Next Steps:**

* Summary of key learnings
* Further resources for advanced SAR data processing and analysis.